<a href="https://colab.research.google.com/github/dolmani38/BLE-signal-collision-simulation/blob/main/simulation_0625.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BLE Signal collision simulation

* 2 signal의 충돌 확률을 고려한다.
* Signal은 5000ms 간격, 2ms 동안 송출
* 2ms 내 3개 channel로 각 1회 총 3회 순차적 송출
* 이에 따른 2 signal의 충돌 확률은 2/5000 * (1/1.4)^3 으로 생각 가능
* 여기서 1/1.4는 동일 channel이 충돌할 확률 

* 이제 